# Automatic Code Generation for Nonlinear Model Predictive Control (NMPC)

## Import modules

In [ ]:
from sympy import *
from codegen_modules import basic_symbolic_func as symfunc
from codegen_modules import generate_cpp_codes as gencpp
init_printing()

## Set dimx, dimu, dimc
dimx: dimension of the state  
dimu: dimension of the control input  
dimc: dimension of the constraints

In [ ]:
dimx = 4
dimu = 2
dimc = 0

## Generate t, x, u, and lmd, necessary variables for the formulation, and basic functions
t: time parameter  
x: state vector  
u: control input vector  
lmd: Lagrange multiplier vector for the state equation

In [ ]:
t, x, u, lmd = symfunc.generateBasicVars(dimx, dimu, dimc)

## Define user variables used in the state function here
Define a scalar variable whose name is "var" as  
`var = Symbol('var') `

Define scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`var_1, var_2, ..., var_n = symbols('var_1, var_2, ..., var_n')`

Define a vector whose name is "vec" and dimension is dim_vec as  
`vec = symbols(f'vec[0:{dim_vec}]')`

In [ ]:
# Define user variables used in the state function here
m1, m2, l1, l2, d1, d2, J1, J2, g = symbols('m1, m2, l1, l2, d1, d2, J1, J2, g')

# You can also define functions used  in the state function or in the cost funciton 

## Define weight parameters used in the stage cost and the terminal cost
q: weight array for the state in the stage cost  
r: weight array for the control input in the stage cost  
q_terminal: weight array for the state in the terminal cost  
x_ref: the reference value of the state

In [ ]:
q = symbols(f'q[0:{dimx}]')
r = symbols(f'r[0:{dimu+dimc}]')
q_terminal = symbols(f'q_terminal[0:{dimx}]')
x_ref = symbols(f'x_ref[0:{dimx}]')

## Define the state function, constraints, stage cost, terminal cost
fxu: state equation  
Cxu: constraisnts  
L: stage cost  
phi: terminal cost  
Note: array indices start with 0

In [ ]:
# Define the state equation
fxu = [x[2], 
        x[3],
        -(sin(x[0]+x[1])*d2*g*m2+g*(d1*m1+l1*m2)*sin(x[0])-2*d2*(x[2]+(1/2)*x[3])*l1*x[3]*m2*sin(x[1])-u[0])/(2*d2*m2*l1*cos(x[1])+d1**2*m1+d2**2*m2+l1**2*m2+J1+J2), 
        ((1/2)*g*d2*l1*m2*(d1*m1+l1*m2)*sin(x[0]-x[1])-(1/2)*d2**2*g*l1*m2**2*sin(x[0]+2*x[1])-(d1**2*m1-(1/2)*d1*l1*m1+(1/2)*l1**2*m2+J1)*m2*g*d2*sin(x[0]+x[1])-l1**2*m2**2*d2**2*(x[2]**2+x[2]*x[3]+(1/2)*x[3]**2)*sin(2*x[1])-l1*m2*d2*(((x[2]+x[3])**2*d2**2+x[2]**2*l1**2)*m2+(d1**2*m1+J1+J2)*x[2]**2+2*J2*x[2]*x[3]+J2*x[3]**2)*sin(x[1])+g*((1/2)*d2**2*l1*m2**2+(d1*d2**2*m1+J2*l1)*m2+J2*d1*m1)*sin(x[0])+2*l1*m2*(u[1]-(1/2)*u[0])*d2*cos(x[1])+((1/2)*(2*u[1]-2*u[0])*d2**2+l1**2*u[1])*m2+(u[1]-u[0])*J2+u[1]*(d1**2*m1+J1))/((2*d2*m2*l1*cos(x[1])+d1**2*m1+d2**2*m2+l1**2*m2+J1+J2)*(d2**2*m2+J2))]

# Define the constraints (if dimc > 0)
Cxu = []

# Define the stage cost
L = sum(q[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2 + sum(r[i] * u[i]**2 for i in range(dimu))/2 

# Define the terminal cost
phi = sum(q_terminal[i]*(x[i] - x_ref[i])**2 for i in range(dimx))/2

## Generate  optimality conditions
hx: partial derivartive of the hamiltonian with respect to x  
hu: partial derivartive of the hamiltonian with respect to u

In [ ]:
if(dimc > 0):
    hamiltonian = L + symfunc.dotProduct(lmd, fxu) + sum(u[dimu+i] * Cxu[i] for i in range(dimc))
else:
    hamiltonian = L + symfunc.dotProduct(lmd, fxu) 

phix = symfunc.diffScalarFunc(phi, x)
hx = symfunc.diffScalarFunc(hamiltonian, x)
hu = symfunc.diffScalarFunc(hamiltonian, u)

## Symplify phix, hx, and hu
Note: if hx and hu is too complicated, it takes too much time to simplify them

In [ ]:
phix = simplify(phix)
hx = simplify(hx)
hu = simplify(hu)

## Set Parameters
set all parameters used in the state equation and the cost function

In [ ]:
# scalar parameters
scalar_params = [[m1, 0.2], [m2, 0.7], [l1, 0.3], [l2, 0.3], [d1, 0.15], [d2, 0.257], [J1, 0.006], [J2, 0.051], [g, 9.80665]]

# array parameters
array_params = [['q', dimx, '{1, 1, 0.1, 0.1}'], ['r', dimu, '{0.1,0.1}'], ['q_terminal', dimx, '{1, 1, 0.1, 0.1}'], ['x_ref', dimx, '{M_PI, 0, 0, 0}']]

## Generate C++ Codes
generate nmpc_model.hpp and nmpc_model.cpp

In [ ]:
gencpp.generateCpp(dimx, dimu, dimc, fxu, Cxu, phix, hx, hu)
gencpp.generateHpp(dimx, dimu, dimc, scalar_params, array_params)